In [1]:
import subprocess

sim_status={}


def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, rx1FRx2):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --useRx1ParamsForRx2ClassC="+f"{rx1FRx2}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os


def run_scenario(traffic_type, realistic_model, device_class, scenario_name, rx1FRx2):
    
    sim_status[scenario_name] = {"success":0, "fail":0}
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, rx1FRx2)
            futures.append(future)



    for future in futures:
        try:
            data = json.loads(future.result())
            if float(data["ULPDR"]["Total"])>0:
                data_all.append(data)
                print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
                sim_status[scenario_name]['success'] += 1
        except:
            print("One of the runs failed")
            sim_status[scenario_name]['fail'] += 1
        
    test_folder_name = 'test_scenario_rx1_for_rx2_class_c'
    test_sub_folder = str(rx1FRx2)

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []


In [7]:

threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", False)))

In [8]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", False)))


threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", False)))

In [9]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", True)))


threads.append(threading.Thread(target=run_scenario,args=(0, False, 'B', "ULPDR_UNCONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'B', "ULPDR_CONFIRMED_B", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'B', "ULPDR_MIXED_B", True)))

In [10]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status Rx1_f_Rx2', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time)+str("\n")+str(sim_status), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_CONFIRMED_B, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 1
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_B, Simualtion with device count: 2
ULPDR_CONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
ULPDR_MIXED_C, Simualtio

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

ULPDR_UNCONFIRMED_C, Simualtion with device count: 20
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=17 --dataUpType=0 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=11 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_B, Simualtion with device count: 13
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=13 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 17
ULPDR_MIXED_C, Simualtion with device count: 32
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceTyp

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=20 --dataUpType=0 --realisticModel=False"
ULPDR_UNCONFIRMED_C, Simualtion with device count: 25
ULPDR_UNCONFIRMED_B, Simualtion with device count: 17
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=B --appPeriod=600 --simulationTime=800 --nDevices=13 --dataUpType=0 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 20
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=17 --dataUpType=2 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevi

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
0.9523809523809523 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
0.9782608695652174 L: 0.0575
0.96 L: 0.0625
0.9552238805970149 L: 0.08375
0.963855421686747 L: 0.10375
One of the runs failed
0.9916666666666667 L: 0.15
0.9724137931034482 L: 0.18125
0.9832402234636871 L: 0.22375
0.9956709956709957 L: 0.28875
0.9964285714285714 L: 0.35
0.985207100591716 L: 0.4225
0.9952267303102625 L: 0.52375
0.984313725490196 L: 0.6375
0.9861111111111112 L: 0.81
0.9592356687898089 L: 0.98125
0.9629248197734295 L: 1.21375
0.9291736930860034 L: 1.4825
0.8903654485049833 L: 1.88125
0.8590568060021436 L: 2.3325
0.8067737733391229 L: 2.87875
0.7447698744769874 L: 3.585
0.6901092743065285 L: 4.46125
0.6484571428571428 L: 5.46875
0.5651056014692378 L: 6.80625


Computing performance metrics...
Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
0.9705882352941176 L: 0.085
1.0 L: 0.10375
1.0 L: 0.11875
1.0 L: 0.1475
1.0 L: 0.1875
1.0 L: 0.23
1.0 L: 0.29125
1.0 L: 0.3625
1.0 L: 0.42875
0.9929742388758782 L: 0.53375
0.9864077669902913 L: 0.64375
0.9861751152073732 L: 0.81375
0.9634760705289672 L: 0.9925
0.9713993871297242 L: 1.22375
0.943744752308984 L: 1.48875
0.914738929279577 L: 1.89125
0.8723972237052856 L: 2.34125
0.8155507559395249 L: 2.89375
0.7477414871438499 L: 3.5975
0.6524168762224085 L: 4.47375
0.5387074674583238 L: 5.47375
0.47001650467632494 L: 6.81625
1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 

Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
0.9803921568627451 L: 0.06375
0.9852941176470589 L: 0.085
0.9879518072289156 L: 0.10375
0.9895833333333334 L: 0.12
1.0 L: 0.15125
0.9867549668874173 L: 0.18875
0.9698492462311558 L: 0.24875
0.9917695473251029 L: 0.30375
0.967741935483871 L: 0.3875
0.977961432506887 L: 0.45375
0.9777777777777777 L: 0.5625
0.9504424778761061 L: 0.70625
0.9614835948644793 L: 0.87625
0.936694021101993 L: 1.06625
0.918233082706767 L: 1.33
0.9134396355353075 L: 1.64625
0.8500303582270795 L: 2.05875
0.7809015996122152 L: 2.57875
0.7474469756480754 L: 3.1825
0.6874606175173283 L: 3.9675
0.6051491205709916 L: 4.90375
0.530874430169913 L: 6.0325
0.4571190674437968 L: 7.50625
3311.048718214035
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
